In [1]:
# Delete, google colab only stuff
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/AOMIC

/content/drive/My Drive/AOMIC


In [3]:
# Same downsampler as before
from tensorflow.keras.layers import Conv3D, MaxPooling3D, InputLayer, Dense, Flatten
from tensorflow.keras.models import Sequential

pooler = Sequential()
pooler.add(MaxPooling3D(input_shape = (160, 256, 256, 1)))

In [4]:
import pandas as pd
import numpy as np
participants = pd.read_csv('participants.tsv', sep='\t')
participants

,participant_id,age,sex,handedness,BMI,education_level,background_SES,IST_fluid,IST_memory,IST_crystallised,IST_intelligence_total,BAS_drive,BAS_fun,BAS_reward,BIS,NEO_N,NEO_E,NEO_O,NEO_A,NEO_C,STAI_T,sexual_attraction_M,sexual_attraction_F,gender_identity_M,gender_identity_F,religious_upbringing,religious_now,religious_importance
0,sub-0001,22.00,female,right,23,medium,2.0,77.0,49.0,33.0,159.0,13,12,16,16,28,50,50,39,47.0,44.0,7.0,1.0,1.0,7.0,no,yes,2.0
1,sub-0002,21.75,female,right,20,medium,5.5,97.0,63.0,39.0,199.0,14,15,19,20,30,46,47,31,45.0,31.0,7.0,1.0,2.0,7.0,no,no,NaN
2,sub-0003,25.25,female,right,31,high,3.0,122.0,67.0,38.0,227.0,9,12,14,20,34,40,41,45,29.0,40.0,6.0,3.0,1.0,6.0,no,no,NaN
3,sub-0004,22.50,female,right,20,high,5.0,149.0,69.0,52.0,270.0,12,10,18,22,29,36,33,36,44.0,32.0,6.0,2.0,1.0,7.0,yes,no,NaN
4,sub-0005,22.25,male,right,23,high,4.5,112.0,57.0,43.0,212.0,9,13,17,20,27,44,35,40,44.0,23.0,1.0,7.0,6.0,1.0,no,no,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,sub-0924,22.25,male,right,21,medium,3.0,136.0,56.0,54.0,246.0,9,14,20,22,40,39,47,38,32.0,56.0,2.0,6.0,4.0,4.0,no,no,NaN
924,sub-0925,25.25,male,right,30,medium,4.0,64.0,37.0,49.0,150.0,15,15,15,13,28,41,48,34,40.0,44.0,1.0,7.0,7.0,1.0,no,no,NaN
925,sub-0926,20.75,male,right,22,high,2.0,84.0,44.0,33.0,161.0,12,10,20,20,27,44,46,41,45.0,30.0,NaN,NaN,NaN,NaN,yes,yes,5.0
926,sub-0927,24.25,female,right,35,medium,2.5,98.0,57.0,35.0,190.0,12,10,16,23,35,32,44,45,41.0,38.0,7.0,2.0,1.0,7.0,no,no,NaN


In [5]:
# Same loading code as before
import os
import nibabel as nb

all_brains = 0
titles = os.listdir('structural_brains')

# I only want to explain these first 10 brains here
# Yes, the brains are from the training set. However, we're just looking for preliminary answers
# so I don't think that's too much of an issue
for brain in titles[:10]: 
  print(brain)
  if all_brains is 0:
    all_brains = pooler.predict(np.expand_dims(np.expand_dims(nb.load('structural_brains/' + brain).get_fdata(), axis=0), axis=-1))
  else:
    all_brains = np.concatenate([all_brains, pooler.predict(np.expand_dims(np.expand_dims(nb.load('structural_brains/' + brain).get_fdata(), axis=0), axis=-1))], axis=0)


sub-0001_run-1_T1w_brain.nii.gz
sub-0002_run-1_T1w_brain.nii.gz
sub-0003_run-1_T1w_brain.nii.gz
sub-0004_run-1_T1w_brain.nii.gz
sub-0005_run-1_T1w_brain.nii.gz
sub-0006_run-1_T1w_brain.nii.gz
sub-0007_run-1_T1w_brain.nii.gz
sub-0008_run-1_T1w_brain.nii.gz
sub-0009_run-1_T1w_brain.nii.gz
sub-0010_run-1_T1w_brain.nii.gz


In [6]:
# Integrated gradients solution copy-pasted from GitHub. 
# Probably better as a module, but I was working on fixing it in notebooks
# I made some changes to make it work, hopefully it works on your machine without any modifications as well

################################################################
# Implemented by Naozumi Hiranuma (hiranumn@uw.edu)            #
#                                                              #
# Keras-compatible implmentation of Integrated Gradients       # 
# proposed in "Axiomatic attribution for deep neuron networks" #
# (https://arxiv.org/abs/1703.01365).                          #
#                                                              #
# Keywords: Shapley values, interpretable machine learning     #
################################################################

from __future__ import division, print_function
import numpy as np
from time import sleep
import sys
import tensorflow.keras.backend as K

from tensorflow.keras.models import Model, Sequential

'''
Integrated gradients approximates Shapley values by integrating partial
gradients with respect to input features from reference input to the
actual input. The following class implements the paper "Axiomatic attribution
for deep neuron networks".
'''

# This eager execution allows this to work on higher tensorflow/keras versions!
from tensorflow.compat.v1 import disable_eager_execution
disable_eager_execution()
class integrated_gradients:
    # model: Keras model that you wish to explain.
    # outchannels: In case the model are multi tasking, you can specify which output you want explain .
    def __init__(self, model, outchannels=[], verbose=1):
    
        #get backend info (either tensorflow or theano)
        self.backend = K.backend()
        
        #load model supports keras.Model and keras.Sequential
        if isinstance(model, Sequential):
            self.model = model# .model
        elif isinstance(model, Model):
            self.model = model
        else:
            print("Invalid input model")
            return -1
        
        #load input tensors
        self.input_tensors = []
        for i in self.model.inputs:
            self.input_tensors.append(i)
        # The learning phase flag is a bool tensor (0 = test, 1 = train)
        # to be passed as input to any Keras function that uses 
        # a different behavior at train time and test time.
        self.input_tensors.append(K.learning_phase())
        
        #If outputchanels are specified, use it.
        #Otherwise evalueate all outputs.
        self.outchannels = outchannels
        if len(self.outchannels) == 0: 
            if verbose: print("Evaluated output channel (0-based index): All")
            if K.backend() == "tensorflow":
                self.outchannels = range(self.model.output.shape[1])
            elif K.backend() == "theano":
                self.outchannels = range(self.model.output._keras_shape[1])
        else:
            if verbose: 
                print("Evaluated output channels (0-based index):")
                print(','.join([str(i) for i in self.outchannels]))
                
        #Build gradient functions for desired output channels.
        self.get_gradients = {}
        if verbose: print("Building gradient functions")
        
        # Evaluate over all requested channels.
        for c in self.outchannels:
            # Get tensor that calculates gradient
            if K.backend() == "tensorflow":
                gradients = self.model.optimizer.get_gradients(self.model.output[:, c], self.model.input)
            if K.backend() == "theano":
                gradients = self.model.optimizer.get_gradients(self.model.output[:, c].sum(), self.model.input)
                
            # Build computational graph that computes the tensors given inputs
            self.get_gradients[c] = K.function(inputs=self.input_tensors, outputs=gradients)
            
            # This takes a lot of time for a big model with many tasks.
            # So lets print the progress.
            if verbose:
                sys.stdout.write('\r')
                sys.stdout.write("Progress: "+str(int((c+1)*1.0/len(self.outchannels)*1000)*1.0/10)+"%")
                sys.stdout.flush()
        # Done
        if verbose: print("\nDone.")
            
                
    '''
    Input: sample to explain, channel to explain
    Optional inputs:
        - reference: reference values (defaulted to 0s).
        - steps: # steps from reference values to the actual sample (defualted to 50).
    Output: list of numpy arrays to integrated over.
    '''
    def explain(self, sample, outc=0, reference=False, num_steps=5, verbose=0):
        
        # Each element for each input stream.
        samples = []
        numsteps = []
        step_sizes = []
        
        # If multiple inputs are present, feed them as list of np arrays. 
        if isinstance(sample, list):
            #If reference is present, reference and sample size need to be equal.
            if reference != False: 
                assert len(sample) == len(reference)
            for i in range(len(sample)):
                if reference == False:
                    _output = integrated_gradients.linearly_interpolate(sample[i], False, num_steps)
                else:
                    _output = integrated_gradients.linearly_interpolate(sample[i], reference[i], num_steps)
                samples.append(_output[0])
                numsteps.append(_output[1])
                step_sizes.append(_output[2])
        
        # Or you can feed just a single numpy arrray. 
        elif isinstance(sample, np.ndarray):
            _output = integrated_gradients.linearly_interpolate(sample, reference, num_steps)
            samples.append(_output[0])
            numsteps.append(_output[1])
            step_sizes.append(_output[2])
            
        # Desired channel must be in the list of outputchannels
        assert outc in self.outchannels
        if verbose: print("Explaning the "+str(self.outchannels[outc])+"th output.")
            
        # For tensorflow backend
        _input = []
        for s in samples:
            _input.append(s)
        _input.append(0)
        print(len(_input))

        if K.backend() == "tensorflow": 
            gradients = self.get_gradients[outc](_input)
        elif K.backend() == "theano":
            gradients = self.get_gradients[outc](_input)
            if len(self.model.inputs) == 1:
                gradients = [gradients]

        explanation = []
        for i in range(len(gradients)):
            _temp = np.sum(gradients[i], axis=0)
            explanation.append(np.multiply(_temp, step_sizes[i]))
           
        # Format the return values according to the input sample.
        if isinstance(sample, list):
            return explanation
        elif isinstance(sample, np.ndarray):
            return explanation[0]
        return -1

    
    '''
    Input: numpy array of a sample
    Optional inputs:
        - reference: reference values (defaulted to 0s).
        - steps: # steps from reference values to the actual sample.
    Output: list of numpy arrays to integrate over.
    '''
    @staticmethod
    def linearly_interpolate(sample, reference=False, num_steps=50):
        # Use default reference values if reference is not specified
        if reference is False: reference = np.zeros(sample.shape);

        # Reference and sample shape needs to match exactly
        assert sample.shape == reference.shape

        # Calcuated stepwise difference from reference to the actual sample.
        ret = np.zeros(tuple([num_steps] +[i for i in sample.shape]))
        for s in range(num_steps):
            ret[s] = reference+(sample-reference)*(s*1.0/num_steps)

        return ret, num_steps, (sample-reference)*(1.0/num_steps)

In [7]:
# Load the predictor trained by classify.ipynb
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input
sequential_model = load_model('predictor.h5')



In [8]:
# Explanation code. Extremely computationally expensive.
# Num_steps may have to be lowered even further if it won't run on your computer
# The low num_steps may be a cause of the blocky artifacts as well.
# (as the name suggests, the integrated_gradients is actually a definite integral
# so more steps should be more accurate)

explainer = integrated_gradients(sequential_model)#model)
explained = []
for i in all_brains:
  explained.append(explainer.explain(i, num_steps=5)) #If the ordinary session has enough RAM, the TPU will work to run this i think. but the GPU will crash :(

Evaluated output channel (0-based index): All
Building gradient functions
Progress: 100.0%
Done.
2


KeyboardInterrupt: ignored

In [ ]:
# Plot sagittal slice of these explained images
import matplotlib.pyplot as plt
for i, _ in enumerate(explained):
  plt.imshow(explained[i][40, :, :, 0]**2, cmap='Reds', alpha=0.7)
  plt.imshow(all_brains[i][40,:,:,0], cmap='Greys', alpha=0.5)
  plt.show()

In [ ]:
# Load explanations+possibly downsampled outputs to output files
# nibabel naturally shifts images upon loading (I think), so even if you don't downsample
# you'll have to rely on this code

if not os.path.exists('brain_samples'):
  os.mkdir('brain_samples')
if not os.path.exists('heatmaps'):
  os.mkdir('heatmaps')
for i, brain in enumerate(titles):
  print(brain)
  img_loaded = nb.load('structural_brains/' +brain)
  ni_img = nb.Nifti1Image(explained[i]**2, img_loaded.affine)
  nb.save(ni_img, 'heatmaps/heatmap'+str(i)+'.nii.gz')

  ni_img = nb.Nifti1Image(all_brains[i], img_loaded.affine)
  nb.save(ni_img, 'brain_samples/brain_sample' + str(i) +'.nii.gz')